# Modif CSV

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r"C:\Users\krish\ai-power-converter\machine-learning\simulation_results_dataset_1000.csv")

# Specify the columns you want to remove
columns_to_remove = ["No", "RL", "ESR", "ESL"]  # Specify the columns you want to remove

# Drop the specified columns
df_truncated = df.drop(columns=columns_to_remove)

# Save the truncated DataFrame back to a CSV file
df_truncated.to_csv("modified_dataset.csv", index=False)


# Scalarization

In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the CSV file
df = pd.read_csv(r"C:\Users\krish\ai-power-converter\machine-learning\modified_dataset.csv")

# Select only the input features (L, C, fsw)
input_features = df[["L", "C", "fsw"]]

# Apply scalarization to input features
scaler = MinMaxScaler()
input_scaled = scaler.fit_transform(input_features)

# Create a DataFrame with the scaled input features
df_input_scaled = pd.DataFrame(input_scaled, columns=["L_scaled", "C_scaled", "fsw_scaled"])

# Concatenate the scaled input features with the remaining columns
df_processed = pd.concat([df_input_scaled, df.drop(columns=["L", "C", "fsw"])], axis=1)

# Save the DataFrame with scaled input features to a new CSV file
df_processed.to_csv("processed_dataset.csv", index=False)


# Machine Learning

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv(r"C:\Users\krish\ai-power-converter\machine-learning\modified_dataset.csv")  # Replace "your_dataset.csv" with the path to your dataset

# Select input features (L, C, fsw)
X = df[['L', 'C', 'fsw']].values

# Scale input features to range [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Output features
# y = df[['Pl_s1', 'Pl_s2', 'Pl_L_Cu', 'Pl_L_Fe', 'Pl_C', '∆Vo%', '∆IL%']].values
y = df[["delta_current_percentage", "delta_voltage_percentage", "switching_loss", "capacitor_loss", "inductor_copper_loss"]].values
# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=(3,)),  # Scalarization layer
    tf.keras.layers.Dense(64, activation='relu'),          # Hidden layer 1
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(64, activation='relu'),          # Hidden layer 2
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(64, activation='relu'),          # Hidden layer 3
    tf.keras.layers.BatchNormalization(),                  # Batch normalization
    tf.keras.layers.Dense(7)                               # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_scaled, y, epochs=50, batch_size=32)

# Optionally, save the model
model.save("power_converter_model.h5")


KeyboardInterrupt: 